In [1]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(30, 10),
            nn.ReLU(),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.linear_relu_stack(x)

In [2]:
model = NeuralNetwork()
model

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=30, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

In [3]:
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y=True)
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).int()

train_X, test_X, train_y, test_y = \
    train_test_split(X, y, test_size=0.10, shuffle=True)

train_data = TensorDataset(train_X, train_y)
train_loader = DataLoader(train_data, batch_size=50)

test_data = TensorDataset(test_X, test_y)
test_loader = DataLoader(test_data, batch_size=len(test_data))

loss = nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

for epoch in range(1):
    train_accuracies = []
    for X, y in tqdm(train_loader, unit="batch"):
        preds = model(X)
        l = loss(preds, y.float())
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        train_accuracies += [100 * torch.mean(
            ((preds > 0.5).int() == y).float()).item()]
    
    print(f"Epoch: {epoch}, \
              Training accuracy: {np.mean(train_accuracies)}")

  0%|          | 0/11 [00:00<?, ?batch/s]/home/ioannis/.conda/envs/ml/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/ioannis/.conda/envs/ml/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([12, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 11/11 [00:00<00:00, 140.70batch/s]

Epoch: 0,               Training accuracy: 61.848484115167096


In [4]:
for X, y in test_loader:
    preds = model(X)
    test_accuracy = 100 * torch.mean(
            ((preds > 0.5).int() == y).float()).item()
    print(f"Test accuracy: {test_accuracy}")

Test accuracy: 68.42105388641357
